# OpenAPI.NET Notebook Sample

In this sample we will use OpenAPI.NET package to access cTrader Open API, and show a trading account real time stats.

Let's start by installing the Nuget package:

In [ ]:
#r "nuget: Spotware.OpenAPI.Net"

Then we will add the OpenAPI.NET usings:

In [ ]:
using OpenAPI.Net;
using OpenAPI.Net.Auth;
using OpenAPI.Net.Helpers;

Please create a credentials.json file in current directory of notebook file and fill it with your Open API credentials:

```json
{
  "ClientId": "your app client id",
  "Secret": "your app secret",
  "AccessToken": "your access token, create one by using your app playground on open API site",
}
```

We will load your created credentials file:

In [ ]:
using System.Text.Json;
using System.Text.Json.Serialization;
using System.IO;

public record Credentials(string ClientId, string Secret, string AccessToken);

private async Task<Credentials> GetCredentials()
{
    using var stream = File.OpenRead("credentials.json");
    return  await JsonSerializer.DeserializeAsync<Credentials>(stream);
}

var credentials = await GetCredentials();

Console.WriteLine(credentials);

We got the API credentials, now lets create a method for getting OpenAPI.NET client, this client will first autorize our API application and then trading account:

In [ ]:
using System.Reactive.Linq;
using Google.Protobuf;

private async Task<OpenClient> GetClient(Mode mode)
{
    var host = ApiInfo.GetHost(mode);

    var client = new OpenClient(host, ApiInfo.Port, TimeSpan.FromSeconds(15));

    client.Subscribe(_ => {}, exception => Console.WriteLine($"Exception: {exception}"));
    client.OfType<ProtoErrorRes>().Subscribe(error => Console.WriteLine($"Exception: {error}"));

    bool isAppAuthroized = false;

    void OnAppAuthorized(ProtoOAApplicationAuthRes response) => isAppAuthroized = true;
    
    client.OfType<ProtoOAApplicationAuthRes>().Subscribe(OnAppAuthorized);
    
    await client.Connect();
    
    var applicationAuthReq = new ProtoOAApplicationAuthReq
    {
        ClientId = credentials.ClientId,
        ClientSecret = credentials.Secret,
    };
    
    await client.SendMessage(applicationAuthReq, applicationAuthReq.PayloadType);

    var waitStartTime = DateTime.Now;

    while (isAppAuthroized is false && DateTime.Now - waitStartTime < TimeSpan.FromSeconds(2))
    {
        await Task.Delay(100);
    }

    if (isAppAuthroized is false) throw new InvalidOperationException("Account authorization failed");

    return client;
}

After create a method to an API client we need to get all the authorized accounts for your access token:

In [ ]:
// Please set one of the authorized trading accounts number
// We will use it for future requests
const long AccountNumber = 3397885;

if (AccountNumber == 0) throw new InvalidOperationException("Please first set the account number");

private async Task<ProtoOAGetAccountListByAccessTokenRes> GetAccounts()
{
    var request = new ProtoOAGetAccountListByAccessTokenReq()
    {
        AccessToken = credentials.AccessToken
    };

    ProtoOAGetAccountListByAccessTokenRes result = null;

    void OnResponse(ProtoOAGetAccountListByAccessTokenRes response) => result = response;

    using var client = await GetClient(Mode.Live);

    client.OfType<ProtoOAGetAccountListByAccessTokenRes>().Subscribe(OnResponse);

    await client.SendMessage(request, request.PayloadType);

    await Task.Delay(1000);

    if (result is null) throw new InvalidOperationException($"Couldn't get response");

    return result;
}

var accounts = await GetAccounts();

var account = accounts.CtidTraderAccount.First(account => account.TraderLogin == AccountNumber);

Console.WriteLine($"Account ID is: {account.CtidTraderAccountId}");

To use an account you have to first authroize it, otherwise you will not be able to send any request related to that account, let's create method for authorized an account: 

In [ ]:
private async Task<ProtoOAAccountAuthRes> AuthorizeAccount(OpenClient client, long accountId)
{
    var request = new ProtoOAAccountAuthReq()
    {
        AccessToken = credentials.AccessToken,
        CtidTraderAccountId = accountId
    };

    ProtoOAAccountAuthRes result = null;

    void OnResponse(ProtoOAAccountAuthRes response) => result = response;
    
    client.OfType<ProtoOAAccountAuthRes>().Subscribe(OnResponse);

    await client.SendMessage(request, request.PayloadType);

    await Task.Delay(500);

    if (result is null) throw new InvalidOperationException($"Couldn't get response");

    return result;
}

We will use the AuthorizeAccount whenever we created a new client after application authorization.

Now our account and API application is authorized and we can start interacting with the API, let's first create a method for getting a symbol data:

In [ ]:
// Be sure your trading account has a symbol with this name,
// if not change it to another symbol name
const string SymbolName = "EURUSD";

private async Task<ProtoOALightSymbol> GetSymbol(string symbolName)
{
    var request = new ProtoOASymbolsListReq
    {
        CtidTraderAccountId = (long)account.CtidTraderAccountId,
    };

    ProtoOALightSymbol result = null;

    void OnResponse(ProtoOASymbolsListRes response)
    {
        result = response.Symbol.FirstOrDefault(symbol => symbol.SymbolName.Equals(SymbolName, StringComparison.OrdinalIgnoreCase));
    }

    using var client = await GetClient(account.IsLive ? Mode.Live : Mode.Demo);

    await AuthorizeAccount(client, accountId);

    client.OfType<ProtoOASymbolsListRes>().Subscribe(OnResponse);

    await client.SendMessage(request, request.PayloadType);

    await Task.Delay(1000);

    if (result is null) throw new InvalidOperationException($"Couldn't get/find the symbol {SymbolName}");

    return result;
}

var symbol = await GetSymbol(SymbolName);
Console.WriteLine(symbol);

Next we will get the above symbol daily trend bars or OHLC price data data from API:

In [ ]:
private async Task<ProtoOAGetTrendbarsRes> GetTrendbars(long symbolId, DateTimeOffset from, DateTimeOffset to)
{
    var request = new ProtoOAGetTrendbarsReq()
    {
        Period = ProtoOATrendbarPeriod.D1,
        CtidTraderAccountId = (long)account.CtidTraderAccountId,
        SymbolId = symbolId,
        FromTimestamp = from.ToUnixTimeMilliseconds(),
        ToTimestamp = to.ToUnixTimeMilliseconds()
    };

    ProtoOAGetTrendbarsRes result = null;

    void OnResponse(ProtoOAGetTrendbarsRes response) => result = response;

    using var client = await GetClient(account.IsLive ? Mode.Live : Mode.Demo);

    await AuthorizeAccount(client, accountId);

    client.OfType<ProtoOAGetTrendbarsRes>().Subscribe(OnResponse);

    await client.SendMessage(request, request.PayloadType);

    await Task.Delay(1000);

    if (result is null) throw new InvalidOperationException($"Couldn't get response");

    return result;
}

// We get one year of daily bars
var trendbars = await GetTrendbars(symbol.SymbolId, DateTimeOffset.UtcNow.AddYears(-1), DateTimeOffset.UtcNow);

Console.WriteLine(trendbars)

What about getting account open positions and orders? let's do it:

In [ ]:
private async Task<ProtoOAReconcileRes> GetOrdersAndPositions()
{
    var request = new ProtoOAReconcileReq()
    {
        CtidTraderAccountId = (long)account.CtidTraderAccountId,
    };

    ProtoOAReconcileRes result = null;

    void OnResponse(ProtoOAReconcileRes response) => result = response;

    using var client = await GetClient(account.IsLive ? Mode.Live : Mode.Demo);

    await AuthorizeAccount(client, accountId);

    client.OfType<ProtoOAReconcileRes>().Subscribe(OnResponse);

    await client.SendMessage(request, request.PayloadType);

    await Task.Delay(1000);

    if (result is null) throw new InvalidOperationException($"Couldn't get response");

    return result;
}

var ordersAndPositions = await GetOrdersAndPositions();

Console.WriteLine(ordersAndPositions)

Please don't use ProtoOAReconcileReq every time you need account open orders/positions, instead use the ExecutionEvent and use ProtoOAReconcileReq only once for getting the already open orders/positions.

All of the above methods return data from account, but they don't perform any trading action, let's do some trading, we will create a market order, a limit order, and a stop order:

In [ ]:
public async Task CreateNewOrder(ProtoOAOrderType orderType, long volume, ProtoOATradeSide tradeSide, double limitPrice = 0, double stopPrice = 0)
{
    var request = new ProtoOANewOrderReq
    {
        CtidTraderAccountId = (long)account.CtidTraderAccountId,
        SymbolId = symbol.SymbolId,
        Volume = volume,
        TradeSide = tradeSide,
        OrderType = orderType
    };

    if (request.OrderType == ProtoOAOrderType.Limit)
    {
        request.LimitPrice = limitPrice;
    }

    if (request.OrderType == ProtoOAOrderType.Stop)
    {
        request.StopPrice = stopPrice;
    }

    using var client = await GetClient(account.IsLive ? Mode.Live : Mode.Demo);

    await AuthorizeAccount(client, accountId);

    await client.SendMessage(request, request.PayloadType);

    await Task.Delay(1000);
}

// First market order, change the volume if it doesn't match symbol min volume
// Volume is in units * 100, so 1000 units equal 100000
await CreateNewOrder(ProtoOAOrderType.Market, 100000, ProtoOATradeSide.Buy);
// Limit order, change the price based on current symbol price
await CreateNewOrder(ProtoOAOrderType.Limit, 100000, ProtoOATradeSide.Buy, limitPrice: 1.5);
// Stop order, change the price based on current symbol price
await CreateNewOrder(ProtoOAOrderType.Stop, 100000, ProtoOATradeSide.Buy, stopPrice: 1.5);


Please check the ProtoOANewOrderReq documentation for setting other order properites like stop loss, take profit, stop limit orders, market range orders, etc.

This was our .NET notebook sample, please check our other samples for more future rich apps like our Blazor web assembly sample.

We love to hear from you and see you contribute to OpenAPI.NET repository.